In [1]:
%load_ext Cython

# Integrating fwdpy with fwdpp and libsequence

## Run some simulations

In [2]:
import fwdpy as fp
import fwdpy.fwdpyio as fpio
import multiprocessing as mp
import numpy as np

In [3]:
#We will simulate 10 replicate populations using
#Python's multiprocessing functionality.

def simpop(seed):
    """
    This function does the evolution
    and returns a serialized population
    """
    N=1000
    theta=100.
    nlist=np.array([N]*(10*N),dtype=np.uint32)
    rng = fp.GSLrng(seed)
    nregions=[fp.Region(0,1,1)]
    sregions=[]
    recregions=nregions
    #Simulate 10 populations
    pops = fp.evolve_regions(rng,1,N,nlist,theta/(4.*float(N)),
                             0.,theta/(4.*float(N)),
                             nregions,sregions,recregions)
    #return the population as a binary string
    return fpio.serialize(pops[0])

#Run 10 sims via a multiprocessing pool
seeds=[(i) for i in np.random.randint(0,42000000,10)]
#Auto-determine degree of parallelism
P=mp.Pool()
#Retrieve each simulated population as a binary-encoded
#string
serialized_pops = [i for i in P.imap(simpop,seeds)]
#Close the pool and block until the sims are done:
P.close()
P.join()
#Turn our strings back into populations:
pops = fpio.deserialize_singlepops(serialized_pops)


## Calculate nucleotide diversity for a sample from each simulated replicate

The next block of code is pure Python.  The function `pi_from_libsequence` uses [pylibseq](https://molpopgen.github.io/pylibseq) to calculate nucleotide diversity in a sample of size $n=100$.  The Python package pylibseq is based on the C++11 library [libsequence](https://molpopgen.github.io/libsequence)

In [4]:
import libsequence.polytable as pt
import libsequence.summstats as st

def pi_using_pylibseq(rng,x,nsam):
    s=fp.ms_sample(rng,x,nsam)
    d=pt.SimData()
    d.assign(s)
    ad=st.PolySIM(d)
    return ad.thetapi()

Behind the scenes, [Cython](http://www.cython.org) is used to pass the simulated population's data back to functions from both the [fwdpp](http://molpopgen.github.io) and [libsequence](https://molpopgen.github.io/libsequence) libraries.

fwdpy exposes a lot of the [fwdpp](http://molpopgen.github.io) API to you via Cython, and [pylibseq](https://molpopgen.github.io/pylibseq) gives you access to what you need from [libsequence](https://molpopgen.github.io/libsequence) to analyze simulation output.

Let's re-implement `py_using_pylibseq` ourselves, so we can see some of what we can do:

### Calculate nucleotide diversity using fwdpy's Cython APIs

In [5]:
fwdpy_includes = fp.get_includes()
fwdpp_includes = fp.get_fwdpp_includes()

In [6]:
%%cython --cplus --compile-args=-std=c++11 -I $fwdpy_includes -I $fwdpp_includes -l sequence -l gsl -l gslcblas

from fwdpy.fwdpy cimport *

#sample_single is an alias for KTfwd::sample from 
#fwdpp/sugar/sampling.hpp.  It is for sampling
#from single-deme, single-region simulations
from fwdpy.fwdpp cimport sample_single
from cython.operator cimport dereference as deref
cdef pi_from_singlepop_cpp(const gsl_rng * r,const singlepop_t * pop, const unsigned nsam):
    s = sample_single[singlepop_t](r,deref(pop),nsam,True)
    pi = 0.
    for i in s:
        ones = float(i[1].count(b'1'))
        pi += 2.0*ones*(float(nsam)-ones)
    pi /= (float(nsam)*float(nsam-1))
    return pi

def pi_using_cython(GSLrng rng,Spop pop,nsam):
    return pi_from_singlepop_cpp(rng.thisptr.get(),pop.pop.get(),nsam)

In [7]:
rng=fp.GSLrng(100)
%timeit -n10 pi = [pi_using_pylibseq(rng,i,50) for i in pops]

10 loops, best of 3: 34.8 ms per loop


In [8]:
rng=fp.GSLrng(100)
%timeit -n10 pi = [pi_using_cython(rng,i,50) for i in pops]

10 loops, best of 3: 20.6 ms per loop


### Quick note on performance

We beat libsequence for the case of calculating a single summary statistic.  This result will not hold in general--libsequence's `PolySIM` (pylibseq's `CppPolySIM`) actually goes over the data twice.  The first time is to pre-process mutation counts at all sites.  The second time is when `thetapi` is called and the stored allele counts are used to calculate the sum of site heterozygosity.  There is an additional inefficiency when using pylibseq due to copying the data (`s`) into libsequence's `SimData` object (`d`).

Further, recall that $\pi$ is part of many other statistics.  libsequence knows this, and its classes will store $\pi$ after the first time it has been calculated, making calculating $\pi$ and Tajima's $D$ and Fay and Wu's $H$ faster by re-using the stored value.

In short, stick with libsequence routines (exposed via pylibseq) for all of the usual reasons to prefer existing libraries over hand-rolled code:

1. they are well-tested
2. they are fast
3. you can still use custom code if you really know you'll only ever be calculating $\pi$.

## Accessing selected and neutral mutations in a sample separately 

The next block of code shows how to get the SFS for segregating variation separately for neutral and selected variants.  We use the SFS function to get $\pi$ separately for the two classes of variants.

In [9]:
%%cython --cplus --compile-args=-std=c++11 -I $fwdpy_includes -I $fwdpp_includes -l sequence -l gsl -l gslcblas

from fwdpy.fwdpy cimport *
#sample_single is an alias for KTfwd::sample from 
#fwdpp/sugar/sampling.hpp.  It is for sampling
#from single-deme, single-region simulations
from fwdpy.fwdpp cimport sample_separate_single,sample_t
from cython.operator cimport dereference as deref
from libcpp.utility cimport pair
from libcpp.string cimport string as cppstring

cdef extern from "<utility>" namespace "std" nogil:
    pair[T,U] make_pair[T,U](T &,U &)

cdef extern from "<algorithm>" namespace "std" nogil:
    unsigned count[ITR,T](ITR,ITR,T)

ctypedef cppstring.const_iterator citr

cdef vector[double] sfs(const sample_t & s) except+:
    cdef vector[double] rv
    rv.resize(s[0].second.size()-1,0)
    cdef size_t i=0,ones
    cdef char derived = b'1'
    while i < s.size():
        ones=count(<citr>s[i].second.begin(),<citr>s[i].second.end(),derived)
        if ones > 0 and ones < s.size(): #skip fixations
            rv[ones-1]+=1
        i+=1
    return rv
    
cdef double pi(const sample_t & s) except+:
    if s.empty() is True:
        return 0.
    cdef vector[double] sample_sfs = sfs(s)
    cdef double pi = 0.
    cdef double nsam = sample_sfs.size()+1
    j=1
    for i in sample_sfs:
        if i>0:
            pi += <double>i*<double>(j)*(nsam-<double>j)
        j+=1
    pi *= 2.0
    pi /= nsam*(nsam-1.)
    return pi    

cdef pi_neutral_selected_cpp(const gsl_rng * r,const singlepop_t * pop, const unsigned nsam) except+:
    cdef sep_sample_t s = sample_separate_single[singlepop_t](r,deref(pop),nsam,True)
    return make_pair[double,double](<double>pi(s.first),<double>pi(s.second))    

def pi_neutral_selected(GSLrng rng,Spop pop,nsam):
    return pi_neutral_selected_cpp(rng.thisptr.get(),pop.pop.get(),nsam)

In [10]:
rng=fp.GSLrng(100)
pins=[pi_neutral_selected(rng,i,50) for i in pops]

In [11]:
pins[0]

(103.99755102040817, 0.0)

In [12]:
rng=fp.GSLrng(100)
pi_using_pylibseq(rng,pops[0],50)

103.99755102040827

As an _interface_, our Cython code is a bit sub-par.  As written, it is tough to get an SFS out for a sample and then get $\pi$ for the same sample.  There are a few alternative possibilities, such as returning the SFS and then defining another function to get $\pi$ from an SFS, perhaps using `numpy`.  I'll leave that as an excercise.

Another possibility is to write functions that sample a pre-defined set of individuals.  This set could be randomly determined in the Python environment, or it could be defined non-randomly:

In [17]:
%%cython --cplus --compile-args=-std=c++11 -I $fwdpy_includes -I $fwdpp_includes -l sequence -l gsl -l gslcblas

from fwdpy.fwdpy cimport *

from fwdpy.fwdpp cimport sample_separate_single_ind,sample_t
from cython.operator cimport dereference as deref
from libcpp.utility cimport pair
from libcpp.string cimport string as cppstring

cdef extern from "<algorithm>" namespace "std" nogil:
    unsigned count[ITR,T](ITR,ITR,T)

ctypedef cppstring.const_iterator citr

cdef vector[double] sfs(const sample_t & s) except+:
    cdef vector[double] rv
    rv.resize(s[0].second.size()-1,0)
    cdef size_t i=0,ones
    cdef char derived = b'1'
    while i < s.size():
        ones=count(<citr>s[i].second.begin(),<citr>s[i].second.end(),derived)
        if ones > 0 and ones < s.size(): #skip fixations
            rv[ones-1]+=1
        i+=1
    return rv

def sfs_sep_from_sample(Spop pop,list individuals, removeFixed=True):
    cdef sep_sample_t s = sample_separate_single_ind[singlepop_t](deref(pop.pop.get()),individuals,removeFixed)
    return (sfs(s.first),sfs(s.second))

def pi_sep_from_sfs(tuple sfs):
    nsam = len(sfs[0])+1
    neutral=0.
    selected=0.
    for i in range(len(sfs[0])):
        neutral += float(sfs[0][i]*i*(nsam-i))
    neutral *= 2.
    neutral /= float(nsam*(nsam-1))
    for i in range(len(sfs[1])):
        selected += float(sfs[1][i]*i*(nsam-i))
    selected *= 2.
    selected /= float(nsam*(nsam-1))
    return (neutral,selected)

Note that this example avoids the need for the C++ `make_pair` function by defining the C++ types in the Python function and simply returning a tuple.